In [1]:
!python -V

Python 3.10.6


In [7]:
import numpy as np
import pandas as pd
from src.data.read import load_file_from_s3
from src.data.read import ID_COLUMN
from src.models import predict

In [3]:
predict.get_model()

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 21cf301e4e7f459bae86218626159897

In [4]:
from src.clients import client

predict_client = client.PredictClient('http://localhost:9000')


In [5]:
passenger_X = {"Age": 23, "Sex": 'female', "Pclass": 2, "Embarked": 'C', "SibSp": 1, "Parch": 1, "Fare": 9}

predict_client.lambda_predict(passenger_X)

'[{"PassengerId": -1, "predictions": 0.8866254687309265, "decisions": 1}]'

In [6]:
lambda_json_s3_path = "s3://mlflow-enkidupal-experiments/data/external/test/test.csv"

response = predict_client.lambda_predict_s3path(lambda_json_s3_path)
response


'[{"PassengerId": 892, "predictions": 0.17178867757320404, "decisions": 0}, {"PassengerId": 893, "predictions": 0.4923032522201538, "decisions": 0}, {"PassengerId": 894, "predictions": 0.1256335973739624, "decisions": 0}, {"PassengerId": 895, "predictions": 0.1516686975955963, "decisions": 0}, {"PassengerId": 896, "predictions": 0.4442468285560608, "decisions": 0}, {"PassengerId": 897, "predictions": 0.1707412302494049, "decisions": 0}, {"PassengerId": 898, "predictions": 0.7031752467155457, "decisions": 1}, {"PassengerId": 899, "predictions": 0.2966756820678711, "decisions": 0}, {"PassengerId": 900, "predictions": 0.6736209392547607, "decisions": 1}, {"PassengerId": 901, "predictions": 0.10040461272001266, "decisions": 0}, {"PassengerId": 902, "predictions": 0.1366390585899353, "decisions": 0}, {"PassengerId": 903, "predictions": 0.25172755122184753, "decisions": 0}, {"PassengerId": 904, "predictions": 0.9200294017791748, "decisions": 1}, {"PassengerId": 905, "predictions": 0.16516140

In [9]:
df2 = pd.read_json(response)
df2

,PassengerId,predictions,decisions
0,892,0.171789,0
1,893,0.492303,0
2,894,0.125634,0
3,895,0.151669,0
4,896,0.444247,0
...,...,...,...
413,1305,0.136639,0
414,1306,0.956732,1
415,1307,0.126338,0
416,1308,0.136639,0


In [8]:
passenger_X = {"Age": 23, "Sex": 'female', "Pclass": 2, "Embarked": 'C', "SibSp": 1, "Parch": 1, "Fare": 9}

test_df = predict.create_features(passenger_X)

test_df


,Age,Sex,Pclass,Embarked,SibSp,Parch,Fare
0,23,female,2,C,1,1,9


In [9]:
from src.data.read import load_file_from_s3
from src.features.build_features import get_all_columns, preprocess_test, get_all_test_columns

model = predict.get_model()
preprocessor = predict.get_preprocessor()

lambda_json_s3_path = "s3://mlflow-enkidupal-experiments/data/external/test/test.csv"
test_df = predict.create_features_for_s3_path(lambda_json_s3_path)
#test_df.head()

first_column_id = test_df.iloc[:, 0]

X_test = preprocess_test(test_df, preprocessor)

print(model)

predictions_np = model.predict(X_test)
predictions = predictions_np.tolist()

decisions = [1 if p >= 0.5 else 0 for p in predictions]
result = {'predictions': list(predictions), 'decisions': list(decisions)}
print(test_df.head())

print(first_column_id.to_numpy())
#print(predictions_np)

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 21cf301e4e7f459bae86218626159897

   PassengerId Pclass                                          Name     Sex  \
0          892      3                              Kelly, Mr. James    male   
1          893      3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894      2                     Myles, Mr. Thomas Francis    male   
3          895      3                              Wirz, Mr. Albert    male   
4          896      3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age SibSp Parch   Ticket     Fare Cabin Embarked  
0  34.5     0     0   330911   7.8292   NaN        Q  
1  47.0     1     0   363272   7.0000   NaN        S  
2  62.0     0     0   240276   9.6875   NaN        Q  
3  27.0     0     0   315154   8.6625   NaN        S  
4  22.0     1     1  3101298  12.2875   NaN        S  
[ 892  893  894  895  896  897  898  899  900  901  902  903  904  905
  90

In [56]:
import pandas as pd

import json
test_df = predict.create_features_for_s3_path(lambda_json_s3_path)
#test_df.head()

# decisions = [1 if p >= 0.5 else 0 for p in predictions]

test_np = test_df.to_numpy()
test_np
first_column_id = test_np[:, 0]
first_column_id

# np.vectorize(my_dict.get)(a)

predictions_np_added = np.c_[first_column_id, predictions_np, np.vectorize(lambda x: 1 if x >= 0.5 else 0)(predictions_np) ]
predictions_np_added
print(predictions_np_added)
#predictions_np_added.dtype.names = ['PassengerId', 'predictions', 'decisions']

predictions_df = pd.DataFrame(data = predictions_np_added, columns = ['PassengerId', 'predictions', 'decisions'])
#predictions_df.set_index('PassengerId', inplace=True)
predictions_df.set_index('PassengerId', inplace=True)
predictions_df.head()


predictions_df.to_json(orient='values')

#jsonStr = json.dumps(predictions_np_added)
#jsonStr

#np.vectorize(lambda x: 1 if x >= 0.5 else 0)(predictions_np)

[[892 0.17178867757320404 0]
 [893 0.4923032522201538 0]
 [894 0.1256335973739624 0]
 ...
 [1307 0.126338392496109 0]
 [1308 0.1366390585899353 0]
 [1309 0.47192907333374023 0]]


'[[0.1717886776,0],[0.4923032522,0],[0.1256335974,0],[0.1516686976,0],[0.4442468286,0],[0.1707412302,0],[0.7031752467,1],[0.2966756821,0],[0.6736209393,1],[0.1004046127,0],[0.1366390586,0],[0.2517275512,0],[0.9200294018,1],[0.165161401,0],[0.9154147506,1],[0.8670912385,1],[0.1481286585,0],[0.2503266037,0],[0.4666742682,0],[0.4333812594,0],[0.4532499611,0],[0.3271332681,0],[0.9225779176,1],[0.4797621071,0],[0.9381495714,1],[0.1181235909,0],[0.9453588724,1],[0.2503266037,0],[0.2517275512,0],[0.1567103714,0],[0.165161401,0],[0.1749505401,0],[0.4568122625,0],[0.371658206,0],[0.5038584471,1],[0.1788357198,0],[0.5101059675,1],[0.4912091196,0],[0.1516686976,0],[0.2559947073,0],[0.2723839283,0],[0.3062547743,0],[0.1339553446,0],[0.8946087956,1],[0.9154147506,1],[0.1516686976,0],[0.2892401218,0],[0.1409783661,0],[0.9412252307,1],[0.4681448936,0],[0.3139845729,0],[0.1469233632,0],[0.8920474052,1],[0.8684867024,1],[0.1553034484,0],[0.1404999346,0],[0.1663540453,0],[0.1532315463,0],[0.1205342934,0

In [32]:
type(predictions_np_added)

numpy.ndarray

In [12]:

DEFAULT_TARGET = 'Survived'
DEFAULT_CATEGORICAL = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
DEFAULT_NUMERICAL = ['Fare', 'Age']
ID_COLUMN = 'PassengerId'


DEFAULT_ALL_COLUMNS = [ID_COLUMN] + DEFAULT_CATEGORICAL + DEFAULT_NUMERICAL
DEFAULT_TRAIN_ALL_COLUMNS = DEFAULT_CATEGORICAL + DEFAULT_NUMERICAL

DEFAULT_ALL_COLUMNS

['PassengerId', 'Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch', 'Fare', 'Age']